# Import Packages

In [ ]:
import numpy as np
import pandas as pd
import keras
from os import listdir
from os.path import join
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# Spliting

In [ ]:
size = (128, 128)
cats_tr = [join('Data/Train/Cats', f) for f in listdir('Data/Train/Cats')]
dogs_tr = [join('Data/Train/Dogs', f) for f in listdir('Data/Train/Dogs')]
cats_te = [join('Data/Test/Cats', f) for f in listdir('Data/Test/Cats')]
dogs_te = [join('Data/Test/Dogs', f) for f in listdir('Data/Test/Dogs')]
dogs_train_x = np.array([img_to_array(load_img(image).resize(size)) for image in dogs_tr])
dogs_train_y = np.array([1 for image in dogs_tr])
cats_train_x = np.array([img_to_array(load_img(image).resize(size)) for image in cats_tr])
cats_train_y = np.array([0 for image in cats_tr]) #0 for cats
dogs_test_x = np.array([img_to_array(load_img(image).resize(size)) for image in dogs_te])
dogs_test_y = np.array([1 for image in dogs_te]) #1 for dogs
cats_test_x = np.array([img_to_array(load_img(image).resize(size)) for image in cats_te])
cats_test_y = np.array([0 for image in cats_te])
x_train = np.append(cats_train_x, dogs_train_x, axis=0)
x_test = np.append(cats_test_x, dogs_test_x, axis = 0)
y_train = np.append(cats_train_y, dogs_train_y)
y_test = np.append(cats_test_y, dogs_test_y)
x_train, y_train = shuffle(x_train, y_train, random_state=0)
x_test, y_test = shuffle(x_test, y_test, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=0)


# ResNet without Augment

In [ ]:
base_model_1 = keras.applications.ResNet50(
    weights = './resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
    input_shape = (size[0], size[1], 3),
    include_top = False
)
base_model_1.trainable = False

# Learning without Augment ResNet50

In [ ]:
input_1 = keras.layers.Input(shape=(size[0], size[0], 3))
normaled_1 = keras.layers.BatchNormalization()(input_1)
out_1_1 = base_model_1(normaled_1, training = False)
out_2_1 = keras.layers.Flatten()(out_1_1)
classifier_1 = keras.Sequential([
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_1'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_2'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(1, activation = 'sigmoid', name = 'class'),]
)
output_1 = classifier_1(out_2_1)
model_1 = keras.Model(input_1, output_1)
model_1.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history_1 = model_1.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

# Fine Tuning ResNet without Augmentation

In [ ]:
base_model_1.trainable = False
for layer in range(1, 10):
    base_model_1.layers[-1*layer].trainable = True
model_1.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history_tune_1 = model_1.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

In [ ]:
model_1.evaluate(x_test, y_test, batch_size = 10)

# Augment

In [ ]:
augment = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(factor=1/6.0),
    keras.layers.RandomZoom(height_factor = 0.25, width_factor = 0.25)]
)

# ResNet50 for Augmentation

In [ ]:
base_model = keras.applications.ResNet50(
    weights = './resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
    input_shape = (size[0], size[1], 3),
    include_top = False
)
base_model.trainable = False
base_model.summary()

# Learning with Augment ResNet50

In [ ]:
input = keras.layers.Input(shape=(size[0], size[0], 3))
normaled = keras.layers.BatchNormalization()(input)
out_0 = augment(normaled)
out_1 = base_model(out_0, training = False)
out_2 = keras.layers.Flatten()(out_1)
classifier = keras.Sequential([
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_1'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_2'),
    keras.layers.BatchNormalization(),
    #keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(1, activation = 'sigmoid', name = 'class'),]
)
output = classifier(out_2)
model = keras.Model(input, output)
model.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history = model.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

# Fine Tuning ResNet with Augmentation

In [ ]:
base_model.trainable = False
for layer in range(1, 10):
    base_model.layers[-1*layer].trainable = True
model.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history_tune = model.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

In [ ]:
model.evaluate(x_test, y_test, batch_size = 10)

In [ ]:
resnet_val_binary_accuracy = history_1.history['val_binary_accuracy'] + history_tune_1.history['val_binary_accuracy']
resnet_binary_accuracy = history_1.history['binary_accuracy'] + history_tune_1.history['binary_accuracy']
resnet_val_binary_accuracy_aug = history.history['val_binary_accuracy'] + history_tune.history['val_binary_accuracy']
resnet_binary_accuracy_aug = history.history['binary_accuracy'] + history_tune.history['binary_accuracy']

resnet_val_binary_loss = history_1.history['val_loss'] + history_tune_1.history['val_loss']
resnet_binary_loss = history_1.history['loss'] + history_tune_1.history['loss']
resnet_val_binary_loss_aug = history.history['val_loss'] + history_tune.history['val_loss']
resnet_binary_loss_aug = history.history['loss'] + history_tune.history['loss']

# Accuracy for ResNet50

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (15, 7.5))
ax1.plot(resnet_val_binary_accuracy_aug)
ax1.plot(resnet_binary_accuracy_aug)
ax1.set(xlabel = 'num of Epochs', ylabel = 'Accuracy')
ax1.set_title('Accuracy for Augmented Data and ResNet50')
ax2.plot(resnet_val_binary_accuracy)
ax2.plot(resnet_binary_accuracy)
ax2.set(xlabel = 'num of Epochs', ylabel = 'Accuracy')
ax2.set_title('Accuracy for Unaugmented Data and ResNet50')
ax1.legend(labels=['validation', 'train'])
ax2.legend(labels=['validation', 'train'])
plt.savefig('res_acc')
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (15, 7.5))
ax1.plot(resnet_val_binary_loss_aug)
ax1.plot(resnet_binary_loss_aug)
ax1.set(xlabel = 'num of Epochs', ylabel = 'Loss')
ax1.set_title('Loss for Augmented Data and ResNet50')
ax2.plot(resnet_val_binary_loss)
ax2.plot(resnet_binary_loss)
ax2.set(xlabel = 'num of Epochs', ylabel = 'Loss')
ax2.set_title('Loss for Unaugmented Data and ResNet50')
ax1.legend(labels=['validation', 'train'])
ax2.legend(labels=['validation', 'train'])
plt.savefig('res_loss')
plt.show()

In [ ]:
model_1.save_weights('res_unaug.h5')
model.save_weights('res_aug.h5')

# VGG16 without Augmentation

In [ ]:
base_model_1 = keras.applications.VGG16(
    weights = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
    input_shape = (size[0], size[1], 3),
    include_top = False
)
base_model_1.trainable = False

# Learning without Augment VGG16

In [ ]:
input_1 = keras.layers.Input(shape=(size[0], size[0], 3))
normaled_1 = keras.layers.BatchNormalization()(input_1)
out_1_1 = base_model_1(normaled_1, training = False)
out_2_1 = keras.layers.Flatten()(out_1_1)
classifier_1 = keras.Sequential([
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_1'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_2'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(1, activation = 'sigmoid', name = 'class'),]
)
output_1 = classifier_1(out_2_1)
model_1 = keras.Model(input_1, output_1)
model_1.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history_1 = model_1.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

# Fine Tuning VGG without Augmentation

In [ ]:
base_model_1.trainable = False
for layer in range(1, 3):
    base_model_1.layers[-1*layer].trainable = True
model_1.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history_tune_1 = model_1.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

In [ ]:
model_1.evaluate(x_test, y_test, batch_size = 10)

# VGG16 for Augmentation

In [ ]:
base_model = keras.applications.VGG16(
    weights = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
    input_shape = (size[0], size[1], 3),
    include_top = False
)
base_model.trainable = False

# Learning with Augment VGG16

In [ ]:
input = keras.layers.Input(shape=(size[0], size[0], 3))
normaled = keras.layers.BatchNormalization()(input)
out_0 = augment(normaled)
out_1 = base_model(out_0, training = False)
out_2 = keras.layers.Flatten()(out_1)
classifier = keras.Sequential([
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_1'),
    keras.layers.BatchNormalization(),
    #keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(15, activation = 'relu', name = 'classifier_2'),
    keras.layers.BatchNormalization(),
    #keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(1, activation = 'sigmoid', name = 'class'),]
)
output = classifier(out_2)
model = keras.Model(input, output)
model.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history = model.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

# Fine Tuning VGG with Augmentation

In [ ]:
base_model.trainable = False
for layer in range(1, 3):
    base_model.layers[-1*layer].trainable = True
model.compile(optimizer = keras.optimizers.legacy.SGD(learning_rate = 0.1, momentum = 0.9, decay = 0.002), loss = 'binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy()])
history_tune = model.fit(x_train, y_train, batch_size = 10, epochs = 25, validation_data = (x_valid, y_valid), shuffle = True)

In [ ]:
model.evaluate(x_test, y_test, batch_size = 10)

In [ ]:
vgg_val_binary_accuracy = history_1.history['val_binary_accuracy'] + history_tune_1.history['val_binary_accuracy']
vgg_binary_accuracy = history_1.history['binary_accuracy'] + history_tune_1.history['binary_accuracy']
vgg_val_binary_accuracy_aug = history.history['val_binary_accuracy'] + history_tune.history['val_binary_accuracy']
vgg_binary_accuracy_aug = history.history['binary_accuracy'] + history_tune.history['binary_accuracy']

vgg_val_binary_loss = history_1.history['val_loss'] + history_tune_1.history['val_loss']
vgg_binary_loss = history_1.history['loss'] + history_tune_1.history['loss']
vgg_val_binary_loss_aug = history.history['val_loss'] + history_tune.history['val_loss']
vgg_binary_loss_aug = history.history['loss'] + history_tune.history['loss']

# Accuracy for VGG

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (15, 7.5))
ax1.plot(vgg_val_binary_accuracy)
ax1.plot(vgg_binary_accuracy)
ax1.set(xlabel = 'num of Epochs', ylabel = 'Accuracy')
ax1.set_title('Accuracy for Unaugmented Data and VGG16')
ax2.plot(vgg_val_binary_accuracy_aug)
ax2.plot(vgg_binary_accuracy_aug)
ax2.set(xlabel = 'num of Epochs', ylabel = 'Accuracy')
ax2.set_title('Accuracy for augmented Data and VGG16')
ax1.legend(labels=['validation', 'train'])
ax2.legend(labels=['validation', 'train'])
plt.savefig('vgg_acc')
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (15, 7.5))
ax1.plot(vgg_val_binary_loss)
ax1.plot(vgg_binary_loss)
ax1.set(xlabel = 'num of Epochs', ylabel = 'Loss')
ax1.set_title('Loss for Unaugmented Data and VGG16')
ax2.plot(vgg_val_binary_loss_aug)
ax2.plot(vgg_binary_loss_aug)
ax2.set(xlabel = 'num of Epochs', ylabel = 'Loss')
ax2.set_title('Loss for augmented Data and VGG16')
ax1.legend(labels=['validation', 'train'])
ax2.legend(labels=['validation', 'train'])
plt.savefig('vgg_loss')
plt.show()

In [ ]:
model_1.save_weights('vgg_unaug.h5')
model.save_weights('vgg_aug.h5')